# Train a `bioLORD` model with `developing human immune across tissue` for `bioLORD` (B-cells)

The data was generated by Suo et al.[[1]](https://www.science.org/doi/full/10.1126/science.abo0510) and downloaded from [Lymphoid cells](https://cellgeni.cog.sanger.ac.uk/developmentcellatlas/fetal-immune/PAN.A01.v01.raw_count.20210429.LYMPHOID.embedding.h5ad). <br>
The complete dataset contains a cross-tissue single-cell atlas of developing human immune cells across prenatal hematopoietic, lymphoid, and nonlymphoid peripheral organs. This includes over 900,000 cells from which we identified over 100 cell states.

[[1] Suo, Chenqu, Emma Dann, Issac Goh, Laura Jardine, Vitalii Kleshchevnikov, Jong-Eun Park, Rachel A. Botting et al. "Mapping the developing human immune system across organs." Science (2022): eabo0510.](https://www.science.org/doi/full/10.1126/science.abo0510)


In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import os
import sys
sys.path.append("/cs/usr/bar246802/bar246802/SandBox2023/biolord_immune_bcells/utils") # add utils
sys.path.append("/cs/usr/bar246802/bar246802/SandBox2023/biolord") # set path)

In [78]:
import numpy as np
import pandas as pd
from os.path import exists
import torch
import umap.plot
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from cluster_analysis import *
from formatters import *
import colorcet as cc

In [79]:
import mplscience
mplscience.set_style()

plt.rcParams['legend.scatterpoints'] = 1

In [95]:
arr_n_latent_attribute_categorical = np.concatenate((np.arange(3, 5, 1), np.arange(5, 31, 5)))
arr_reconstruction_penalty = [1e1, 1e2, 1e3]
arr_unknown_attribute_penalty = [1e-2, 1e-1, 1e1]
arr_unknown_attribute_noise_param = [1e-2, 1e-1, 1e1]
print(len(arr_n_latent_attribute_categorical))
print(arr_n_latent_attribute_categorical)
print(len(arr_n_latent_attribute_categorical) * len(arr_reconstruction_penalty) * len(arr_unknown_attribute_penalty) * len(arr_unknown_attribute_noise_param))

8
[ 3  4  5 10 15 20 25 30]
216


## Set parameters

In [80]:
DATA_DIR = "../data/"
SAVE_DIR = "../output/"
FIG_DIR = "../figures/"
LOGS_CSV = SAVE_DIR + "trained_models_scores.csv"

In [81]:
df = pd.read_csv(DATA_DIR + "trained_models_scores.csv")
#     sns.set_style("whitegrid")
plt.style.use('default')
sns.set_style(style='white')
sns.set_theme(style='white')
sns.color_palette("colorblind")
attribues  = list(set(df['n_latent_attribute_categorical']))
palette = sns.cubehelix_palette(start=.5, rot=-.75, dark=0.45, n_colors=len(attribues))
attributes_color_map = dict(zip(np.unique(attribues), palette))
# reshape the d dataframe suitable for statsmodels package
cols = ["row_index", "attribute", "score_name", "score", "n_clusters", "n_latent_attribute_categorical",
        "reconstruction_penalty",
        "unknown_attribute_penalty",
        "unknown_attribute_noise_param",
        "id_"]
# keys_cols = ["row_index", "attribute", "score_name", "n_clusters", "n_latent_attribute_categorical",
#         "reconstruction_penalty",
#         "unknown_attribute_penalty",
#         "unknown_attribute_noise_param",
#         "id_"]
# ragelar_cols = ["score"]
# df_melt = pd.melt(df.reset_index(), id_vars=keys_cols, value_vars=ragelar_cols)

for score_name in list(set(df['score_name'])):
    fig, axs = plt.subplots(1, 2, figsize=(14, 7), gridspec_kw={"width_ratios": [1, 1]})
    col = row = 0
    title = f'n_latent_attribute_categorical vs. score of metric: {score_name}'
    for attribute in list(set(df['attribute'])):
        df_score = df[(df['score_name'] == score_name) & (df['attribute'] == attribute)]
        sns.boxplot(x='n_latent_attribute_categorical', y='score', data=df_score,
                    palette=attributes_color_map, ax=axs[col]).set(title=title)
        sns.swarmplot(x="n_latent_attribute_categorical", y="score", s=3, data=df_score, color='#7d0013', ax=axs[col])
        axs[col].set_title(f'attribute: {attribute}')
        axs[col].set_xlabel("n_latent_attribute_categorical")
        axs[col].set_ylabel("Score")
#             axs[col].grid(False)
        axs[col].set_facecolor('white')
        col += 1
    fig.suptitle(title, fontsize=14)
    plt.savefig(FIG_DIR + f'{score_name}_score_boxplot.png', format="png", dpi=300)
    plt.show()
    plt.close()

In [123]:
import scipy.stats as stats
for score_name in list(set(df['score_name'])):
    for attribute in list(set(df['attribute'])):        
        df_score = df[(df['score_name'] == score_name) & (df['attribute'] == attribute)]
        print('***********************************************************************')
        print(f"ANOVA for metric: {score_name} and attribute: {attribute}")
        grouped_data = [group['score'] for _, group in df_score.groupby('n_latent_attribute_categorical')]
        fvalue, pvalue = stats.f_oneway(*grouped_data)
        print(f'fvalue = {fvalue}, pvalue = {pvalue}')
        if pvalue < 0.05:
            print(f"pValue is less than 0.05 hence we reject H0. \n There exists a difference btw n_latent_attribute_categorical for metric: {score_name} and attribute {attribute}")
        else:
            print(f"pValue is not less than 0.05 hence we can't reject H0. \n There is no difference btw n_latent_attribute_categorical for metric: {score_name} and attribute {attribute}")
        # get ANOVA table as R like output
        import statsmodels.api as sm
        from statsmodels.formula.api import ols

        # Ordinary Least Squares (OLS) model
        model = ols('score ~ C(n_latent_attribute_categorical)', data=df_score).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        print(anova_table)
        # ANOVA table using bioinfokit v1.0.3 or later (it uses wrapper script for anova_lm)
        from bioinfokit.analys import stat
        res = stat()
        res.anova_stat(df=df_score, res_var='score', anova_model='score ~ C(n_latent_attribute_categorical)')
        res.anova_summary
        print('***********************************************************************')
        print('\n')

# note: if the data is balanced (equal sample size for each group), Type 1, 2, and 3 sums of squares
# (typ parameter) will produce similar results.

***********************************************************************
ANOVA for metric: Adjusted Rand Index and attribute: organ
fvalue = 3.940062648358961, pvalue = 0.0014301197842703384
pValue is less than 0.05 hence we reject H0. 
 There exists a difference btw n_latent_attribute_categorical for metric: Adjusted Rand Index and attribute organ
                                     sum_sq    df         F   PR(>F)
C(n_latent_attribute_categorical)  0.691736   7.0  3.940063  0.00143
Residual                           1.429598  57.0       NaN      NaN
***********************************************************************


***********************************************************************
ANOVA for metric: Adjusted Rand Index and attribute: celltype
fvalue = 15.983060945849827, pvalue = 2.0327241348509e-11
pValue is less than 0.05 hence we reject H0. 
 There exists a difference btw n_latent_attribute_categorical for metric: Adjusted Rand Index and attribute celltype
              

                                     sum_sq    df          F        PR(>F)
C(n_latent_attribute_categorical)  1.815797   7.0  13.323995  4.858051e-10
Residual                           1.109711  57.0        NaN           NaN
***********************************************************************




In [115]:
# we will use bioinfokit (v1.0.3 or later) for performing tukey HSD test
# check documentation here https://github.com/reneshbedre/bioinfokit
from bioinfokit.analys import stat
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
for score_name in list(set(df['score_name'])):
    for attribute in list(set(df['attribute'])):        
        print('***********************************************************************')
        df_score = df[(df['score_name'] == score_name) & (df['attribute'] == attribute)]
        print(f" Tukey-kramer for metric: {score_name} and attribute: {attribute}")
    #     print(df_metric)
        # perform multiple pairwise comparison (Tukey's HSD)
        # unequal sample size data, tukey_hsd uses Tukey-Kramer test
        res = stat()
        res.tukey_hsd(df=df_score,
                      res_var='score',
                      xfac_var='n_latent_attribute_categorical',
                      anova_model='score ~ C(n_latent_attribute_categorical)')
        print(res.tukey_summary)
        print('***********************************************************************')
        print('\n')

***********************************************************************
 Tukey-kramer for metric: Adjusted Rand Index and attribute: organ
    group1  group2      Diff     Lower     Upper   q-value   p-value
0        3       4  0.093480 -0.148619  0.335578  1.717923  0.900000
1        3       5  0.018592 -0.230525  0.267710  0.332053  0.900000
2        3      10  0.172410 -0.076708  0.421527  3.079194  0.382131
3        3      15  0.077333 -0.171784  0.326450  1.381149  0.900000
4        3      20  0.085576 -0.163541  0.334694  1.528373  0.900000
5        3      25  0.066180 -0.182938  0.315297  1.181951  0.900000
6        3      30  0.170169 -0.078948  0.419287  3.039184  0.399481
7        4       5  0.074887 -0.167211  0.316986  1.376244  0.900000
8        4      10  0.265889  0.023791  0.507988  4.886388  0.021798
9        4      15  0.170813 -0.071286  0.412911  3.139114  0.356683
10       4      20  0.179056 -0.063043  0.421155  3.290606  0.297720
11       4      25  0.027300 -0.2

    group1  group2      Diff     Lower     Upper   q-value   p-value
0        3       4  0.095668 -0.105191  0.296528  2.119123  0.780963
1        3       5  0.007245 -0.199437  0.213928  0.155964  0.900000
2        3      10  0.058231 -0.148451  0.264914  1.253522  0.900000
3        3      15  0.005945 -0.200738  0.212628  0.127975  0.900000
4        3      20  0.019416 -0.187266  0.226099  0.417968  0.900000
5        3      25  0.134769 -0.071913  0.341452  2.901125  0.460011
6        3      30  0.241135  0.034452  0.447817  5.190813  0.011731
7        4       5  0.088423 -0.112436  0.289283  1.958637  0.846445
8        4      10  0.153900 -0.046960  0.354759  3.408986  0.256201
9        4      15  0.089723 -0.111136  0.290583  1.987437  0.834694
10       4      20  0.076252 -0.124607  0.277111  1.689037  0.900000
11       4      25  0.039101 -0.161759  0.239960  0.866110  0.900000
12       4      30  0.145466 -0.055393  0.346326  3.222180  0.323054
13       5      10  0.065476 -0.14

    group1  group2      Diff     Lower     Upper   q-value   p-value
0        3       4  0.128604 -0.111885  0.369094  2.379243  0.674825
1        3       5  0.102164 -0.145298  0.349625  1.836825  0.896147
2        3      10  0.059921 -0.187541  0.307382  1.077330  0.900000
3        3      15  0.056945 -0.190516  0.304407  1.023835  0.900000
4        3      20  0.001133 -0.246329  0.248594  0.020364  0.900000
5        3      25  0.146150 -0.101312  0.393611  2.627663  0.573461
6        3      30  0.347918  0.100456  0.595379  6.255299  0.001092
7        4       5  0.026441 -0.214049  0.266930  0.489166  0.900000
8        4      10  0.188525 -0.051964  0.429014  3.487807  0.230994
9        4      15  0.071659 -0.168831  0.312148  1.325726  0.900000
10       4      20  0.127472 -0.113018  0.367961  2.358289  0.683375
11       4      25  0.017545 -0.222944  0.258035  0.324599  0.900000
12       4      30  0.219313 -0.021176  0.459803  4.057406  0.098447
13       5      10  0.162084 -0.08

    group1  group2      Diff     Lower     Upper   q-value   p-value
0        3       4  0.011656 -0.028129  0.051440  1.303457  0.900000
1        3       5  0.005587 -0.035351  0.046525  0.607172  0.900000
2        3      10  0.035549 -0.005389  0.076487  3.863516  0.133808
3        3      15  0.016885 -0.024053  0.057823  1.835085  0.896858
4        3      20  0.012110 -0.028828  0.053048  1.316106  0.900000
5        3      25  0.007553 -0.033385  0.048491  0.820870  0.900000
6        3      30  0.030649 -0.010289  0.071587  3.330968  0.283187
7        4       5  0.017242 -0.022542  0.057027  1.928233  0.858849
8        4      10  0.047205  0.007420  0.086990  5.278983  0.009751
9        4      15  0.028541 -0.011244  0.068325  3.191744  0.335074
10       4      20  0.023765 -0.016019  0.063550  2.657719  0.561200
11       4      25  0.004103 -0.035682  0.043887  0.458789  0.900000
12       4      30  0.018994 -0.020791  0.058778  2.124081  0.778940
13       5      10  0.029963 -0.01

In [ ]:
: